In [60]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# Read CSV file
df = pd.read_csv('new_keyword_dataset.csv')
print("Initial DataFrame head:")
print(df.head())

# Fill missing search volume with 0, convert to numeric
df['Avg. monthly searches'] = pd.to_numeric(df['Avg. monthly searches'], errors='coerce').fillna(0)
df.dropna(subset=['Keyword'], inplace=True)
print(f"Total rows after cleaning: {len(df)}")

# Vectorize keywords and cluster (using full dataset)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Keyword'])
kmeans = KMeans(n_clusters=4, random_state=42)
df.loc[:, 'Cluster'] = kmeans.fit_predict(X)

# Define intent function
def assign_intent(keyword):
    keyword = keyword.lower()
    if 'what is' in keyword or 'vs' in keyword or 'programming' in keyword:
        return 'Informational'
    elif 'tools' in keyword or 'test' in keyword or 'testing' in keyword or 'deploy' in keyword or 'automation' in keyword:
        return 'Transactional'
    elif 'best' in keyword or ('software' in keyword and 'tools' not in keyword):
        return 'Commercial'
    else:
        return 'Navigational'

# Remove duplicates and assign intent
df = df.drop_duplicates(subset=['Keyword'])
df.loc[:, 'Intent'] = df['Keyword'].apply(assign_intent)

# Save full dataset with all columns
df.to_csv('new_clustered_keywords.csv', index=False)
print(f"Saved to 'new_clustered_keywords.csv' with {len(df.columns)} columns.")

Initial DataFrame head:
                       Keyword Region  Avg. monthly searches Growth  \
0                    build app    USA                 5000.0     0%   
1  website and app development    USA                 5000.0     0%   
2                   app tester    USA                 5000.0     0%   
3          no code app builder    USA                 5000.0     0%   
4             best app builder    USA                 5000.0   -90%   

  Competition  Competition (indexed value)  Top of page bid (low range)  \
0      Medium                         41.0                         2.94   
1         Low                         10.0                         6.07   
2         Low                         24.0                         2.47   
3      Medium                         42.0                         3.89   
4      Medium                         49.0                         3.51   

   Top of page bid (high range) Start Date   End Date  
0                         12.10   3/1/2024

In [ ]:
# Group by cluster, get top 10 by Avg. monthly searches, and save to CSV
top_10_per_cluster = pd.concat(
    [group.sort_values('Avg. monthly searches', ascending=False).head(10) 
     for _, group in df.groupby('Cluster')]
)

# Print for verification
for cluster, group in top_10_per_cluster.groupby('Cluster'):
    print(f"\nCluster {cluster} (Top 10 by Avg. monthly searches):")
    print(group)

# Save to CSV with all columns
top_10_per_cluster.to_csv('top_10_per_cluster.csv', index=False)
print(f"Saved top 10 keywords per cluster to 'top_10_per_cluster.csv' with {len(top_10_per_cluster.columns)} columns.")


Cluster 0 (Top 10 by Avg. monthly searches):
                                      Keyword Region  Avg. monthly searches  \
171                      backend optimization    USA                50000.0   
134                     openapi specification    USA                50000.0   
138  backend developer vs front end developer    USA                 5000.0   
135                            fastapi python    USA                 5000.0   
141            front end vs backend developer    USA                 5000.0   
142               front vs back end developer    USA                 5000.0   
140            front end developer vs backend    USA                 5000.0   
89                 Automated error correction    USA                  500.0   
137          backend and frontend programming    USA                  500.0   
113               Code generation for backend    USA                  500.0   

    Growth Competition  Competition (indexed value)  \
171     0%         Low       

: 